In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("./model")

In [3]:
import torch
import argparse
import functools

In [4]:
import analyze
from quant import *
from outlier import *
from eval import *
from collections import defaultdict
from pprint import pprint
from modelutils_llama import quantize_model_llama, reorder_model_llama, quantize_model_gptq_llama,  add_act_quant_wrapper_llama
from modelutils_opt import quantize_model_opt, reorder_model_opt, quantize_model_gptq_opt,  add_act_quant_wrapper_opt
from modelutils_mixtral import quantize_model_mixtral, add_act_quant_wrapper_mixtral, reorder_model_mixtral
from parallel_utils import map_layers_to_multi_gpus
from LMClass import LMClass
from eval import pattern_match
from lm_eval import tasks as lm_tasks
from lm_eval import evaluator as lm_evaluator
from datautils import *

In [5]:
torch.set_printoptions(precision=10)

In [6]:
model = torch.load("./saved/llama2-7b_quantized.pth")

In [7]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x QLlamaDecoderLayer(
        (self_attn): QLlamaAttention(
          (q_proj): QLinearLayer(
            (quantizer): Quantizer_GPTQ()
          )
          (k_proj): QLinearLayer(
            (quantizer): Quantizer_GPTQ()
          )
          (v_proj): QLinearLayer(
            (quantizer): Quantizer_GPTQ()
          )
          (o_proj): QLinearLayer(
            (quantizer): Quantizer_GPTQ()
          )
          (rotary_emb): LlamaRotaryEmbedding()
          (act_quant): Quantizer()
          (v_quant): Quantizer()
          (k_quant): Quantizer()
        )
        (mlp): QLlamaMLP(
          (gate_proj): QLinearLayer(
            (quantizer): Quantizer_GPTQ()
          )
          (down_proj): QLinearLayer(
            (quantizer): Quantizer_GPTQ()
          )
          (up_proj): QLinearLayer(
            (quantizer): Quantizer_GPTQ()
          )
      

In [8]:
DEV = torch.device("cuda:0")
dataloader, testloader = get_loaders(
    'wikitext2', seed=0, model="./llama2-7b", seqlen=model.seqlen
)

In [9]:
inputs, outputs = analyze.extract_io(model, testloader, DEV)

100%|██████████| 32/32 [00:22<00:00,  1.42it/s]


In [10]:
quantized_inputs, scales, bases, dequantized_inputs = analyze.get_input_quant_param_dict(inputs)
weights, bias, scales = analyze.get_weight_quant_param_dict(model)

NameError: name 'get_input_quant_param_dict' is not defined

In [ ]:
len(next(iter(inputs.values())))

3

In [ ]:
# torch.save(inputs, "./saved/llama2-7b_inputs.pth")
# torch.save(outputs, "./saved/llama2-7b_outputs.pth")
# torch.save(scales, "./saved/llama2-7b_scales.pth")